In [ ]:
# !git clone https://github.com/tldrafael/DeepLabV3Plus-Pytorch
# !mv DeepLabV3Plus-Pytorch deeplabv3

# !pip install git+https://github.com/huggingface/transformers.git

In [ ]:
# !pip install super_gradients

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from seaborn import color_palette
from PIL import Image
import pandas as pd
from glob import iglob
import re
from datetime import datetime
import random
import cv2
from copy import copy
from transformers import Mask2FormerForUniversalSegmentation
from importlib import reload
from glob import iglob
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from sklearn.manifold import TSNE
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torchvision.transforms as T
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.tensorboard import SummaryWriter
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image

import sys
sys.path.append('../src')
import utils as ut
import dataset as ds

In [ ]:
id2label = ds.IDs('mocamba').id2label
n_classes = len(id2label)
print(id2label)


# long = 256
long = 512
# long = 1024
# long = 1536
# long = 2048

f = long/512
crop_size = (256*f, 352*f) 
pad = int(32 - crop_size[1] % 32)
pad_val = int(32 - ((144/256)*long % 32))
crop_size = tuple(int(a) for a in crop_size)
print(f'long: {long}, crop_size: {crop_size}, pad: {pad}, pad-val: {pad_val}')

T_padval = T.Pad((0, pad_val, 0, 0))

T_crop = T.Compose([
    T.RandomCrop(size=crop_size),
    T.RandomHorizontalFlip(p=.5),
    T.Pad((0,0,pad,0)),
])


train_annotation = f'../data/tidyv01-long{long}/trainpaths.txt'
val_annotation = f'../data/tidyv01-long{long}/valpaths.txt'


train_ds = ut.SimpleDataset(annotation_file=train_annotation, transform=T_crop, transform_target=T_crop)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

val_ds = ut.SimpleDataset(annotation_file=val_annotation, transform=T_padval, transform_target=T_padval)
val_loader = DataLoader(val_ds, batch_size=1)

In [ ]:
# tmp=[]
for inp_im, inp_label in val_loader:
    # tmp.append(inp_label.max())
    break

   

dummy_im = inp_im.clone()[:6]
dummy_label = inp_label.clone()[:6]
dummy_label.unique()
print(dummy_im.shape)

colorizer = ut.TorchColorizer(len(ds.mocamba_classnames)+1)

ims = ut.Normalize.reverse(dummy_im)
labels = colorizer(dummy_label)
alpha = .2
blend = (1-alpha)*ims + alpha*labels

tmp = torch.concat([ims, labels, blend], axis=-2)
tmp = tmp.moveaxis(0,-2).flatten(-2,-1).permute(1,2,0)
tmp = ut.float_to_uint8(tmp.numpy())
Image.fromarray(tmp)

# pre-trained pipelite

In [ ]:

from super_gradients.training import models
from super_gradients.common.object_names import Models


model = models.get(
    model_name=Models.PP_LITE_T_SEG75,
    arch_params={"use_aux_heads": False},
    # num_classes=len(ds.mocamba_classnames),
    pretrained_weights="cityscapes"
)
     

In [ ]:
model.eval().cuda();

pred = model(dummy_im.cuda())
print(pred.shape)
pred = pred.argmax(1, keepdim=True).long()
segmentation_map = pred.cpu()

# colorizer = ut.TorchColorizer(len(ds.mocamba_classnames))
colorizer = ut.TorchColorizer(19)
colorizer.colors = colorizer.colors[np.random.permutation(len(colorizer.colors))]

ims = ut.Normalize.reverse(dummy_im)
labels = colorizer(segmentation_map)
alpha = .2
blend = (1-alpha)*ims + alpha*labels

tmp = torch.concat([ims, labels, blend], axis=-2)
tmp = tmp.moveaxis(0,-2).flatten(-2,-1).permute(1,2,0)
tmp = ut.float_to_uint8(tmp.numpy())
Image.fromarray(tmp)

# Training

In [ ]:
def get_CM_fromloader(dloader, model, n_classes, pad=0, set_='train'):
    
    model.eval()
    CM_abs = np.zeros((n_classes, n_classes), dtype=int)
    for inp_im, inp_label in dloader:
        test_preds = model(inp_im.cuda())[0].argmax(1, keepdim=True).cpu()
        if set_ == "train":
            test_preds = test_preds[..., pad:]
            inp_label = inp_label[..., pad:]
        else:
            test_preds = test_preds[..., pad:,:]
            inp_label = inp_label[..., pad:,:]
        
        for pr_i, y_i in zip(test_preds, inp_label.cpu()):
            CM_abs += ut.get_CM(pr_i, y_i, n_classes)
        
    pred_P = CM_abs.sum(axis=0)
    gt_P = CM_abs.sum(axis=1)
    true_P = np.diag(CM_abs)
    
    CM_iou = true_P / (pred_P + gt_P - true_P)
    miou = np.nanmean(CM_iou)
    return miou, CM_iou, CM_abs
    
    
from super_gradients.training.losses import DiceCEEdgeLoss

criterea = DiceCEEdgeLoss(
    num_classes=len(ds.mocamba_classnames),
    num_aux_heads=3,
    num_detail_heads=0,
    weights=[ 1., 1., 1., 1. ],
    dice_ce_weights=[ 1., 1. ],
    ce_edge_weights=[ .5, .5 ],
    edge_kernel=5
)

In [ ]:

from super_gradients.training import models
from super_gradients.common.object_names import Models


model = models.get(
    model_name=Models.PP_LITE_T_SEG75,
    arch_params={"use_aux_heads": True},
    num_classes=len(ds.mocamba_classnames),
    pretrained_weights="cityscapes"
)

model.cuda();

In [ ]:
opt = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = ut.WarmupLR(opt, n_warmup_max=1000)
warmup_step = 1


time_now = datetime.now().strftime("%Y%m%d_%H%M")
time_now += f'-PAPER-pplite-tidyv01+long{long}'
print(time_now)
logdir = os.path.join('../logs', time_now)
writer = SummaryWriter(log_dir=logdir)
writer.flush()


scaler = torch.cuda.amp.GradScaler()

In [ ]:
val_ds = ut.SimpleDataset(annotation_file=f'../data/mocamba/ds-mocamba-v0.3.4-long{long}/valpaths.txt', transform=T_padval, transform_target=T_padval)
val_loader = DataLoader(val_ds, batch_size=1)

bs_train = 8
train_loader = DataLoader(train_ds, batch_size=bs_train, shuffle=True)

model.cuda();

In [ ]:
fl_resume = True
fl_resume = False


if not fl_resume:
    it = 1
    losses = []
    miou_best = 0
    steps_cur = 0
    lossi = []
       
    lr_cur = opt.param_groups[0]["lr"]
    print(f'lr: {lr_cur:.2e}')
    writer.add_scalars('lr', {'nn': lr_cur}, steps_cur)

    model.eval()
    miou_train = get_CM_fromloader(train_loader, model, n_classes, set_='train')[0]
    miou_val = get_CM_fromloader(val_loader, model, n_classes, set_='val')[0]
    print(f'\tmiou-train: {miou_train:.2f}\tmiou-val: {miou_val:.2f}')
    writer.add_scalars('miou', {'train': miou_train}, steps_cur)
    writer.add_scalars('miou', {'val': miou_val}, steps_cur)


print_every_n = 250
iters_total = 10000
n_gradacc = max(8//bs_train,1) # bs=8
iters_total *= n_gradacc
print_every_n *= n_gradacc
print(f'grad-acc: {n_gradacc}')
print(f'iters_total: {iters_total}')


model.train()
opt.zero_grad(set_to_none=True)
while it <= iters_total:
    
    for inp_im, inp_label in train_loader:
        inp_im = inp_im.cuda()
        inp_label = inp_label[:, 0].cuda()
        
        # Only forward pass goes into the autocast context
        with torch.cuda.amp.autocast(dtype=torch.float16, enabled=False):
            pred = model(inp_im.cuda())
            loss = criterea(pred, inp_label)[0]
            
        scaler.scale(loss).backward()
        # Accumulate gradients
        if it % n_gradacc == 0:
            scaler.step(opt)
            scaler.update()
            opt.zero_grad(set_to_none=True)            
            
        
        lossi.append(loss.item())
        it += 1
        
        if scheduler.fl_warmup and it % warmup_step == 0:
            scheduler.step()
            print(f'lr: {opt.param_groups[0]["lr"]:.2e}')
        
        if it % print_every_n == 0:
            steps_cur += 1

            loss_avg = np.mean(lossi)
            losses.append(loss_avg)

            model.eval()
            miou_train = get_CM_fromloader(train_loader, model, n_classes, pad=pad, set_='train')[0]
            miou_val = get_CM_fromloader(val_loader, model, n_classes, pad=pad_val, set_='val')[0]
            model.train()

            print(
                f'it: {it}\tloss: {loss_avg:.4f}'
                f'\tmiou-train: {miou_train:.3f}',
                f'\tmiou-val: {miou_val:.3f}'
            )

            writer.add_scalars('objective', {'train': loss_avg}, steps_cur)
            writer.add_scalars('miou', {'train': miou_train}, steps_cur)
            writer.add_scalars('miou', {'val': miou_val}, steps_cur)

            
            lossi = []
            
            ckpt_last_path = os.path.join(logdir, 'model.last.pth')
            ut.save_ckpt(ckpt_last_path, model, opt, miou_val, it)

            if miou_val > miou_best:
                miou_best = miou_val
                ckpt_last_path = os.path.join(logdir, 'model.best.pth')
                ut.save_ckpt(ckpt_last_path, model, opt, miou_val, it)

            if it in [5000, 9000]:
                for pg in opt.param_groups:
                    pg['lr'] *= .1
                    
        if it > iters_total:
            break

In [ ]:
# for pg in opt.param_groups:
#     print(pg['lr'])
#     pg['lr'] *= .1
#     print(pg['lr'])
    

In [ ]:
model.eval()
pred = get_hrnet_segmap(dummy_im).cpu()
ims = ut.Normalize.reverse(dummy_im)
# labels = colorizer(dummy_label)
labels = colorizer(pred)
alpha = .2
blend = (1-alpha)*ims + alpha*labels
tmp = torch.concat([ims, labels, blend], axis=-2)
tmp = tmp.moveaxis(0,-2).flatten(-2,-1).permute(1,2,0)
tmp = ut.float_to_uint8(tmp.numpy())
Image.fromarray(tmp)

# Evaluating model

In [ ]:
n_classes = len(ds.mocamba_classnames)

hrnet = models.seg_hrnet_ocr
hrnet.BatchNorm2d_class = hrnet.BatchNorm2d = nn.BatchNorm2d
hrnet = hrnet.get_seg_model(config)

# modelpath = '../logs/20240619_1547-hrnet-res512/model.best.pth'
modelpath = '../logs/20240619_2345-hrnet-res1024-adam/model.best.pth'


state_dict = torch.load(modelpath)['state']
hrnet.load_state_dict(state_dict);
hrnet.eval().cuda()
model = hrnet

In [ ]:
miou, CM_iou, CM_abs = get_CM_fromloader(val_loader, model, n_classes)
print(miou.round(2))
print(CM_iou.round(2))

In [ ]:
# mocamba_classnames = {
#     0: 'background', 1: 'thing-Animals', 2: 'surface-Asphalt', 3: 'sign-Cat-s-Eye', 4: 'damage-Cracks', 5: 'thing-Ego',
#     6: 'surface-Hard-Sand', 7: 'sign-Markings', 8: 'thing-Obstacle', 9: 'thing-People', 10: 'damage-Pothole', 11: 'thing-Retaining-wall',
#     12: 'surface-Soft-Sand', 13: 'surface-Unpaved', 14: 'thing-Vehicles', 15: 'sign-Vertical-Signs', 16: 'surface-Wet-sand'
# }


txt = ''
for v, p in zip(ds.mocamba_classnames.values(), CM_iou):
    txt += f'{v}:\t{round(p*100,2)}\n'

print(txt)

In [ ]:
# fpath = 'all-trainpaths-wider.txt'
# fpath = 'all-valpaths-wider.txt'
# fpath = '../data/mocamba/ds-mocamba-v0.3.4-long1024/trainpaths.txt'
fpath = '../data/mocamba/ds-mocamba-v0.3.4-long1024/valpaths.txt'

colorizer = ut.TorchColorizer(len(ds.mocamba_classnames))


val_ds = ut.SimpleDataset(annotation_file=fpath)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=True)
hrnet.eval()
alpha = .2
nplot = 10

plot_ims = []
for inp_im, inp_label in val_loader:
    pred = get_hrnet_segmap(inp_im).cpu()
    im = ut.Normalize.reverse(inp_im)
    label = colorizer(inp_label)
    
    pred = colorizer(pred)
    blend = (1-alpha)*im + alpha*pred

    tmp = [im, label, pred, blend]
    tmp = [F.pad(x, (0, 1024-x.shape[-1], 0, 0, 0, 0, 0, 0)) for x in tmp]

    tmp = torch.concat([torch.concat(tmp, axis=-1)], axis=-2)
    tmp = tmp.moveaxis(0,-2).flatten(-2,-1).permute(1,2,0)
    tmp = ut.float_to_uint8(tmp.numpy())
    plot_ims.append(tmp)

    if len(plot_ims) >= nplot:
        break


tmp = np.concatenate(plot_ims, axis=0)
Image.fromarray(tmp)

# Compare the worst images performance

In [ ]:
fpath = 'all-trainpaths-wider.txt'
fpath = 'all-valpaths-wider.txt'
fpath = '../data/mocamba/ds-mocamba-v0.3.3-long1024/trainpaths.txt'
# fpath = '../data/mocamba/ds-mocamba-v0.3.3-long1024/valpaths.txt'


with open(fpath, 'r') as f:
    valpaths = f.read().split('\n')[:-1]


ious = []
for p in valpaths[:]:
    im = read_image(p) / 255
    inp = ut.Normalize.forward(im)[None]
    
    pgt = ut.get_gtpath(p)
    label = read_image(pgt)
    if 'RTK' in p:
        label = rtk2mocamba[label.int()]            
    
    pred = m2f(inp.cuda())[0].cpu()
    cur_metric = ut.get_single_image_miou(pred, label, n_classes)
    # cur_metric = (pred == label).float().mean()
    ious.append(cur_metric.item())
    # break


ixs = np.argsort(ious)[:]
samplepaths = np.array(valpaths)[ixs]

In [ ]:
np.array(ious)[ixs].round(2)

In [ ]:
len(samplepaths)

In [ ]:
ix = 3
os.path.basename(samplepaths[-1+start+ix]).split('_jpg')[0].split('_png')[0]

In [ ]:

m2f.eval()
alpha = .2
nplot = 20    

start = 0
plot_ims = []
for p in samplepaths[start:]:
    im = read_image(p) / 255
    inp = ut.Normalize.forward(im)[None]
    
    pgt = ut.get_gtpath(p)
    label = read_image(pgt)
    if 'RTK' in p:
        label = rtk2mocamba[label.int()]            
    label_bkp = label.clone()
    label = colorizer(label[None].long())[0]
    
    pred = m2f(inp.cuda())[0].cpu()[0]
    pred_bkp = pred.clone()
    pred = colorizer(pred[None].long())[0]
    # blend = (1-alpha)*im + alpha*pred
    blend = (1-alpha)*im + alpha*label
    
    tmp = [im, label, pred, blend]
    tmp = [F.pad(x, (0, 1024-x.shape[-1], 0, 0, 0, 0)) for x in tmp]

    tmp = torch.concat([torch.concat(tmp, axis=-1)], axis=-2)
    tmp = tmp.permute(1,2,0)
    tmp = ut.float_to_uint8(tmp.numpy())
    tmp = ut.resize_im(tmp, long=2048)
    plot_ims.append(tmp)

    if len(plot_ims) >= nplot:
        break



tmp = np.concatenate(plot_ims, axis=0)
Image.fromarray(tmp)

In [ ]:
p = samplepaths[2]
im = read_image(p) / 255
inp = ut.Normalize.forward(im)[None]

pgt = ut.get_gtpath(p)
label = read_image(pgt)
if 'RTK' in p:
    label = rtk2mocamba[label.int()]            
pred = m2f(inp.cuda())[0].cpu()[0]

[id2label[x.item()] for x in label.unique()], \
[id2label[x.item()] for x in pred.unique()]

In [ ]:
points = [(400, 420)]
plt.imshow(pred.permute(1,2,0))

for x, y in points:
    plt.scatter(x, y)
    cl_label = label[0, y, x].item()
    cl_pred = pred[0, y, x].item()
    print(id2label[cl_label], id2label[cl_pred])

# Play with t-sne

In [ ]:
for n, _ in hrnet.stage4[-1].named_children():
    print(n)

print('\n\n')
for n, _ in hrnet.named_children():
    print(n)

In [ ]:
L = hrnet.stage4

features = []
hooks = [
    L.register_forward_hook(
        lambda self, input, output: features.append(output)
    )
]

features = []


i = 0
predictions = []
for inp_im, inp_label in tqdm(val_loader):
    with torch.no_grad():
        pred = get_hrnet_segmap(inp_im).cpu()
    # print(features[0][0].shape, inp_im.shape, pred.shape)
    
    pred_as_featsize = T.functional.resize(
        pred, (144, 256), interpolation=T.InterpolationMode.NEAREST)[0,0]
    predictions.append(pred_as_featsize.flatten(0,1))

    i += 1
    # break


for h in hooks:
    h.remove()


predictions = torch.concat(predictions)
print(predictions.shape)
X_feats = [f[0][0].cpu().detach() for f in features]
print(features[0][0].shape)
X_feats = torch.stack(X_feats).permute(0,2,3,1).flatten(0,2)
# X_feats = torch.stack(X_feats).permute(1,0,2,3).flatten(1,3)
print(X_feats.shape)

In [ ]:
n_classes = len(ds.mocamba_classnames)
range_ = np.linspace(0, 1, n_classes)
colors = color_palette('gist_ncar', as_cmap=True)(range_)
colors = colors[np.random.permutation(len(colors))]
colors = torch.tensor(colors)


cl_surfaces = [2, 6, 12, 13, 15]
# # cl_surfaces = [6, 12, 13, 15]
print([ds.mocamba_classnames[c] for c in cl_surfaces])

samples = colors[cl_surfaces][None].float()
# print(samples.shape);
plt.imshow(samples);

In [ ]:
tsne = TSNE(n_jobs=2, perplexity=15)

ixs = np.random.choice(len(X_feats), 4000)
Y = tsne.fit_transform(X_feats[ixs])
# ixs = np.random.choice(len(X_feats[0]), 2000)
# Y = tsne.fit_transform(X_feats[:,ixs].T)

mask = torch.isin(predictions[ixs], torch.tensor(cl_surfaces))
mask = torch.argwhere(mask).ravel().numpy()
mask = np.random.choice(mask, 2000)

cs = colors[predictions[ixs][mask]]

print([ds.mocamba_classnames[c] for c in cl_surfaces])
plt.imshow(samples); plt.show()
plt.scatter(Y[mask, 0], Y[mask, 1], c=cs, alpha=.4); plt.show()
